<a href="https://colab.research.google.com/github/Ishita2patel/CIND820-Project/blob/main/DataScaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install memory_profiler


In [3]:
#Data processing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

#Model performance metrics
from time import process_time
from memory_profiler import profile
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#Feature selection and models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

#Data scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
%load_ext memory_profiler

In [4]:
#Decision Tree Function
def DTClassifier(X_train, y_train, X_test, y_test):
  #Initialize the DecisionTreeClassifier
  tree_raw_imbalanced = DecisionTreeClassifier(criterion = "entropy")

  #Time Measurement
  start_time = process_time()

  #Fit the Classifier to the data
  tree_raw_imbalanced.fit(X_train, y_train)

  #Predict new Data
  y_pred = tree_raw_imbalanced.predict(X_test)

  #Time Measurement
  end_time = process_time()

  #Results
  cr = classification_report(y_test, y_pred)
  cm = confusion_matrix(y_test, y_pred)
  time = end_time - start_time
  print(cr)
  print(cm)
  print(time)
#Random Forest Model
def RFClassifier(X_train, y_train, X_test, y_test):
  #Initialize the Random Forest Classifier
  forest_raw_imbalanced = RandomForestClassifier(n_estimators = 100)

  #Time Measurement
  start_time = process_time()

  #Fit the classifier to the data
  forest_raw_imbalanced.fit(X_train, y_train)

  #Predict new Data
  y_pred = forest_raw_imbalanced.predict(X_test)

  #Time Measurement
  end_time = process_time()

  #Results
  cr = classification_report(y_test, y_pred)
  cm = confusion_matrix(y_test, y_pred)
  time = end_time - start_time
  print(cr)
  print(cm)
  print(time)

In [5]:
#Logistic Regression Model
def LRClassifier(X_train, y_train, X_test, y_test):
  #Initialize the Logistic Regression Classifier
  lr_raw_imbalanced = LogisticRegression(max_iter= 1000)

  #Time Measurement
  start_time = process_time()

  #Fit the classifier to the data
  lr_raw_imbalanced.fit(X_train, y_train)

  #Predict new Data
  y_pred = lr_raw_imbalanced.predict(X_test)

  #Time Measurement
  end_time = process_time()

  #Results
  cr = classification_report(y_test, y_pred)
  cm = confusion_matrix(y_test, y_pred)
  time = end_time - start_time
  print(cr)
  print(cm)
  print(time)


In [6]:
#https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("online_shoppers_intention.csv")
#Identify categorical attributes
categorical_features = ["Month", "OperatingSystems", "Browser", "Region", "TrafficType", "VisitorType", "Weekend"]
df_cat = df[categorical_features]

df_onehot = pd.get_dummies(df, columns = categorical_features, prefix = categorical_features)

#Tranform categorical attributes
label_encoder = LabelEncoder()
df_onehot['Revenue'] = label_encoder.fit_transform(df['Revenue'])

In [7]:
#Specify independent/ dependent values
X = df_onehot.drop(columns = "Revenue")
y = df_onehot["Revenue"]

#Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

Filtered Data
Pearson Correlation Filters

In [8]:
#Correlation of Onehot encoded dataset

corr = df_onehot.corr()

revenue_correlation = corr["Revenue"]
sorted_pearson_correlation = revenue_correlation.abs().sort_values(ascending = False)

sorted_pearson_correlation_df = pd.DataFrame(sorted_pearson_correlation)
SPC_topquantile = sorted_pearson_correlation_df.quantile(0.75)
filtered_df = sorted_pearson_correlation_df[sorted_pearson_correlation_df >= SPC_topquantile]
filtered_df.dropna(inplace = True)
#18 attributes were kept, were in the top quantile

df_pearson = df_onehot[filtered_df.index.tolist()]

#Tranform categorical attributes
label_encoder = LabelEncoder()
df_pearson['Revenue'] = label_encoder.fit_transform(df_pearson['Revenue'])

#Specify independent/ dependent values
X_p = df_pearson.drop(columns = "Revenue")
y_p = df_pearson["Revenue"]

#Split the Data
X_p_train, X_p_test, y_p_train, y_p_test = train_test_split(X_p, y_p, test_size = 0.3)

smote = SMOTE()
X_p_train_smote, y_p_train_smote = smote.fit_resample(X_p_train, y_p_train)

<ipython-input-8-a15d2170359d>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pearson['Revenue'] = label_encoder.fit_transform(df_pearson['Revenue'])


Random Forest Features

In [9]:
#Filters of RF Classifier

rf_classifier = RandomForestClassifier()

#Fit random forest classifier
rf_classifier.fit(X_train_smote, y_train_smote)

feature_importances = rf_classifier.feature_importances_

rf_df = pd.DataFrame({"Feature": X_train_smote.columns, "Importance": feature_importances})

sorted_features = np.argsort(feature_importances)[::-1]

#Sorting features
rf_df_sorted = rf_df.sort_values("Importance", ascending = False)
rf_df_sorted = rf_df_sorted.reset_index(drop = True)
rf_df_sorted

rf_df_sorted.describe()
RF_topquantile = rf_df_sorted['Importance'].quantile(0.75)
RFfiltered_df = rf_df_sorted.loc[rf_df_sorted['Importance'] >= RF_topquantile]
#19 features were kept after keeping the top quartile of results
filtered_attributes_rf = RFfiltered_df.index.tolist()
df_rf = df_onehot[RFfiltered_df["Feature"]]

df_rf["Revenue"] = df_onehot["Revenue"]

#Specify independent/ dependent values
X_rf = df_rf.drop(columns = "Revenue")
y_rf = df_rf["Revenue"]

#Split the Data
X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(X_rf, y_rf, test_size = 0.3)

smote = SMOTE()
X_rf_train_smote, y_rf_train_smote = smote.fit_resample(X_rf_train, y_rf_train)


<ipython-input-9-72b89fc352e0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf["Revenue"] = df_onehot["Revenue"]


RFE Feature Selection

In [10]:
#Filter for features with RFE

df_X_rfe = df_onehot.drop(columns = "Revenue")
df_y_rfe = df_onehot["Revenue"]

# Instantiate the model and RFE selector
model = LogisticRegression(solver = "liblinear")
rfe_selector = RFE(model, n_features_to_select = 20)

# Perform RFE feature selection
selected_features = rfe_selector.fit_transform(df_X_rfe, df_y_rfe)

# Get the mask of selected features
feature_mask = rfe_selector.support_

# Get the ranking of features (optional)
feature_ranking = rfe_selector.ranking_

selected_indices = [i for i, mask in enumerate(feature_mask) if mask]
print("Selected feature indices:", selected_indices)
rfe_ranking_df = pd.DataFrame({"Feature": df_X_rfe.columns, "Rank": feature_ranking})
rfe_ranking_df = rfe_ranking_df.sort_values("Rank")
RFEtop_quartile_value = rfe_ranking_df['Rank'].quantile(0.25)
RFEfiltered_df = rfe_ranking_df[rfe_ranking_df['Rank'] <= RFEtop_quartile_value]
RFEfiltered_df

filtered_attributes_rfe = RFEfiltered_df.index.tolist()
df_rfe = df_onehot[RFEfiltered_df["Feature"]]

#20 features are kept after random forest feature selection
df_rfe["Revenue"] = df_onehot["Revenue"]

#Specify independent/ dependent values
X_rfe = df_rfe.drop(columns = "Revenue")
y_rfe = df_rfe["Revenue"]

#Split the Data
X_rfe_train, X_rfe_test, y_rfe_train, y_rfe_test = train_test_split(X_rfe, y_rfe, test_size = 0.3)

smote = SMOTE()
X_rfe_train_smote, y_rfe_train_smote = smote.fit_resample(X_rfe_train, y_rfe_train)


Selected feature indices: [6, 7, 9, 10, 12, 13, 17, 18, 19, 22, 30, 39, 50, 52, 56, 62, 64, 67, 73, 74]


<ipython-input-10-e8d0f826279d>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rfe["Revenue"] = df_onehot["Revenue"]


In [11]:
#Define columns that need data normalization/ standardization
numeric_features = ['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']

p_numeric_features = ['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration','BounceRates',
       'ExitRates', 'PageValues', 'SpecialDay']

rf_numeric_features = ['Administrative', 'Administrative_Duration',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues']

rfe_numeric_features = ['BounceRates', 'ExitRates', 'SpecialDay']

cat_features = [col for col in X_train if col not in numeric_features]
p_cat_features = [col for col in X_p_train if col not in numeric_features]
rf_cat_features = [col for col in X_rf_train if col not in numeric_features]
rfe_cat_features = [col for col in X_rfe_train if col not in numeric_features]
%memit RFClassifier(X_train_smote, y_train_smote, X_test, y_test)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3137
           1       0.67      0.64      0.65       562

    accuracy                           0.90      3699
   macro avg       0.80      0.79      0.80      3699
weighted avg       0.90      0.90      0.90      3699

[[2963  174]
 [ 205  357]]
3.2941009260000023
peak memory: 322.93 MiB, increment: 19.07 MiB


In [12]:
%memit RFClassifier(X_p_train_smote, y_p_train_smote, X_p_test, y_p_test)

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      3112
           1       0.62      0.72      0.67       587

    accuracy                           0.89      3699
   macro avg       0.79      0.82      0.80      3699
weighted avg       0.89      0.89      0.89      3699

[[2857  255]
 [ 163  424]]
3.2490241080000004
peak memory: 323.01 MiB, increment: 0.07 MiB


In [13]:
%memit RFClassifier(X_rf_train_smote, y_rf_train_smote, X_rf_test, y_rf_test)

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      3110
           1       0.65      0.67      0.66       589

    accuracy                           0.89      3699
   macro avg       0.79      0.80      0.80      3699
weighted avg       0.89      0.89      0.89      3699

[[2900  210]
 [ 195  394]]
2.450852220999998
peak memory: 323.04 MiB, increment: 0.03 MiB


Control Model Performance
Unfiltered Data

In [14]:
%memit DTClassifier(X_train_smote, y_train_smote, X_test, y_test)

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3137
           1       0.53      0.61      0.57       562

    accuracy                           0.86      3699
   macro avg       0.73      0.76      0.74      3699
weighted avg       0.87      0.86      0.86      3699

[[2836  301]
 [ 221  341]]
0.21827646300000225
peak memory: 323.05 MiB, increment: 0.00 MiB


In [15]:
%memit RFClassifier(X_train_smote, y_train_smote, X_test, y_test)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3137
           1       0.67      0.66      0.66       562

    accuracy                           0.90      3699
   macro avg       0.80      0.80      0.80      3699
weighted avg       0.90      0.90      0.90      3699

[[2956  181]
 [ 193  369]]
2.3421089350000024
peak memory: 323.10 MiB, increment: 0.01 MiB


In [16]:
%memit LRClassifier(X_train_smote, y_train_smote, X_test, y_test)

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      3137
           1       0.66      0.46      0.54       562

    accuracy                           0.88      3699
   macro avg       0.78      0.71      0.74      3699
weighted avg       0.87      0.88      0.87      3699

[[3005  132]
 [ 304  258]]
5.663261786000007
peak memory: 331.09 MiB, increment: 7.99 MiB


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pearson correlation features

In [17]:
%memit DTClassifier(X_p_train_smote, y_p_train_smote, X_p_test, y_p_test)

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      3112
           1       0.52      0.63      0.57       587

    accuracy                           0.85      3699
   macro avg       0.72      0.76      0.74      3699
weighted avg       0.86      0.85      0.85      3699

[[2763  349]
 [ 215  372]]
0.21812561400000163
peak memory: 314.60 MiB, increment: 0.05 MiB


In [18]:
%memit RFClassifier(X_p_train_smote, y_p_train_smote, X_p_test, y_p_test)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      3112
           1       0.61      0.74      0.67       587

    accuracy                           0.89      3699
   macro avg       0.78      0.83      0.80      3699
weighted avg       0.90      0.89      0.89      3699

[[2840  272]
 [ 153  434]]
2.8588364190000064
peak memory: 315.09 MiB, increment: 0.48 MiB


In [19]:
%memit LRClassifier(X_p_train_smote, y_p_train_smote, X_p_test, y_p_test)

              precision    recall  f1-score   support

           0       0.93      0.88      0.91      3112
           1       0.51      0.67      0.58       587

    accuracy                           0.85      3699
   macro avg       0.72      0.78      0.74      3699
weighted avg       0.87      0.85      0.86      3699

[[2740  372]
 [ 193  394]]
5.210010787999998
peak memory: 315.10 MiB, increment: 0.01 MiB


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest features

In [20]:
%memit DTClassifier(X_rf_train_smote, y_rf_train_smote, X_rf_test, y_rf_test)

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      3110
           1       0.53      0.61      0.57       589

    accuracy                           0.85      3699
   macro avg       0.73      0.76      0.74      3699
weighted avg       0.86      0.85      0.86      3699

[[2796  314]
 [ 228  361]]
0.1589414139999974
peak memory: 315.12 MiB, increment: 0.00 MiB


In [21]:
%memit RFClassifier(X_rf_train_smote, y_rf_train_smote, X_rf_test, y_rf_test)

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      3110
           1       0.66      0.69      0.67       589

    accuracy                           0.89      3699
   macro avg       0.80      0.81      0.80      3699
weighted avg       0.90      0.89      0.89      3699

[[2899  211]
 [ 183  406]]
2.896500275000001
peak memory: 315.13 MiB, increment: 0.00 MiB


In [22]:
%memit LRClassifier(X_rf_train_smote, y_rf_train_smote, X_rf_test, y_rf_test)

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3110
           1       0.59      0.58      0.59       589

    accuracy                           0.87      3699
   macro avg       0.75      0.75      0.75      3699
weighted avg       0.87      0.87      0.87      3699

[[2868  242]
 [ 245  344]]
4.775792871
peak memory: 315.18 MiB, increment: 0.04 MiB


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Recursive Feature Elimination Features

In [23]:
%memit DTClassifier(X_rfe_train_smote, y_rfe_train_smote, X_rfe_test, y_rfe_test)

              precision    recall  f1-score   support

           0       0.87      0.74      0.80      3102
           1       0.24      0.42      0.31       597

    accuracy                           0.69      3699
   macro avg       0.55      0.58      0.55      3699
weighted avg       0.77      0.69      0.72      3699

[[2305  797]
 [ 346  251]]
0.14826772499999663
peak memory: 315.18 MiB, increment: 0.00 MiB


In [24]:
%memit RFClassifier(X_rfe_train_smote, y_rfe_train_smote, X_rfe_test, y_rfe_test)

              precision    recall  f1-score   support

           0       0.88      0.78      0.83      3102
           1       0.28      0.46      0.35       597

    accuracy                           0.73      3699
   macro avg       0.58      0.62      0.59      3699
weighted avg       0.78      0.73      0.75      3699

[[2410  692]
 [ 325  272]]
2.152745908
peak memory: 342.66 MiB, increment: 27.47 MiB


In [25]:
%memit LRClassifier(X_rfe_train_smote, y_rfe_train_smote, X_rfe_test, y_rfe_test)

              precision    recall  f1-score   support

           0       0.93      0.60      0.73      3102
           1       0.27      0.78      0.41       597

    accuracy                           0.63      3699
   macro avg       0.60      0.69      0.57      3699
weighted avg       0.83      0.63      0.68      3699

[[1867 1235]
 [ 131  466]]
0.3646215459999951
peak memory: 326.19 MiB, increment: 0.03 MiB


Data Normalization

In [26]:
#Subsetting data to scale numeric features

#Control
X_train_smote_NUM = X_train_smote[numeric_features]
X_test_NUM = X_test[numeric_features]

#Pearson correlation features
X_p_train_smote_NUM = X_p_train_smote[p_numeric_features]
X_p_test_NUM = X_p_test[p_numeric_features]

#Random Forest features
X_rf_train_smote_NUM = X_rf_train_smote[rf_numeric_features]
X_rf_test_NUM = X_rf_test[rf_numeric_features]

#Recursive Feature Elimination features
X_rfe_train_smote_NUM = X_rfe_train_smote[rfe_numeric_features]
X_rfe_test_NUM = X_rfe_test[rfe_numeric_features]
nscaler = MinMaxScaler()

#Scaling control
X_train_smote_NUM_N = nscaler.fit_transform(X_train_smote_NUM)
X_train_smote_COMBINED_N = np.concatenate((X_train_smote_NUM_N, X_train_smote[cat_features]), axis = 1)

X_test_NUM_N = nscaler.transform(X_test_NUM)
X_test_COMBINED_N = np.concatenate((X_test_NUM_N, X_test[cat_features]), axis = 1)

#Scaling Pearson correlation features
X_p_train_smote_NUM_N = nscaler.fit_transform(X_p_train_smote_NUM)
X_p_train_smote_COMBINED_N = np.concatenate((X_p_train_smote_NUM_N, X_p_train_smote[p_cat_features]), axis = 1)

X_p_test_NUM_N = nscaler.transform(X_p_test_NUM)
X_p_test_COMBINED_N = np.concatenate((X_p_test_NUM_N, X_p_test[p_cat_features]), axis = 1)

#Scaling Random Forest features
X_rf_train_smote_NUM_N = nscaler.fit_transform(X_rf_train_smote_NUM)
X_rf_train_smote_COMBINED_N = np.concatenate((X_rf_train_smote_NUM_N, X_rf_train_smote[rf_cat_features]), axis = 1)

X_rf_test_NUM_N = nscaler.transform(X_rf_test_NUM)
X_rf_test_COMBINED_N = np.concatenate((X_rf_test_NUM_N, X_rf_test[rf_cat_features]), axis = 1)

#Scaling Recursive Feature Elimination features
X_rfe_train_smote_NUM_N = nscaler.fit_transform(X_rfe_train_smote_NUM)
X_rfe_train_smote_COMBINED_N = np.concatenate((X_rfe_train_smote_NUM_N, X_rfe_train_smote[rfe_cat_features]), axis = 1)

X_rfe_test_NUM_N = nscaler.transform(X_rfe_test_NUM)
X_rfe_test_COMBINED_N = np.concatenate((X_rfe_test_NUM_N, X_rfe_test[rfe_cat_features]), axis = 1)

In [27]:
%memit RFClassifier(X_train_smote_COMBINED_N, y_train_smote, X_test_COMBINED_N, y_test)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3137
           1       0.68      0.65      0.66       562

    accuracy                           0.90      3699
   macro avg       0.81      0.80      0.80      3699
weighted avg       0.90      0.90      0.90      3699

[[2962  175]
 [ 198  364]]
2.678173772000001
peak memory: 351.00 MiB, increment: 24.73 MiB


In [28]:
%memit RFClassifier(X_p_train_smote_COMBINED_N, y_p_train_smote, X_p_test_COMBINED_N, y_p_test)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      3112
           1       0.62      0.73      0.67       587

    accuracy                           0.89      3699
   macro avg       0.78      0.82      0.80      3699
weighted avg       0.89      0.89      0.89      3699

[[2847  265]
 [ 160  427]]
3.1820316040000023
peak memory: 342.43 MiB, increment: 18.68 MiB


In [29]:
%memit RFClassifier(X_rf_train_smote_COMBINED_N, y_rf_train_smote, X_rf_test_COMBINED_N, y_rf_test)

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      3110
           1       0.65      0.69      0.67       589

    accuracy                           0.89      3699
   macro avg       0.79      0.81      0.80      3699
weighted avg       0.89      0.89      0.89      3699

[[2889  221]
 [ 184  405]]
2.538944445999988
peak memory: 342.65 MiB, increment: 0.22 MiB


Control features - Normalized Data

In [30]:
%memit DTClassifier(X_train_smote_COMBINED_N, y_train_smote, X_test_COMBINED_N, y_test)

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3137
           1       0.53      0.61      0.57       562

    accuracy                           0.86      3699
   macro avg       0.73      0.76      0.74      3699
weighted avg       0.87      0.86      0.86      3699

[[2834  303]
 [ 220  342]]
0.2794547490000099
peak memory: 345.46 MiB, increment: 0.08 MiB


In [31]:
%memit RFClassifier(X_train_smote_COMBINED_N, y_train_smote, X_test_COMBINED_N, y_test)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3137
           1       0.67      0.66      0.66       562

    accuracy                           0.90      3699
   macro avg       0.80      0.80      0.80      3699
weighted avg       0.90      0.90      0.90      3699

[[2950  187]
 [ 190  372]]
2.986000734000001
peak memory: 353.90 MiB, increment: 8.62 MiB


In [32]:
%memit LRClassifier(X_train_smote_COMBINED_N, y_train_smote, X_test_COMBINED_N, y_test)

              precision    recall  f1-score   support

           0       0.89      0.98      0.93      3137
           1       0.74      0.31      0.44       562

    accuracy                           0.88      3699
   macro avg       0.82      0.65      0.69      3699
weighted avg       0.87      0.88      0.86      3699

[[3076   61]
 [ 386  176]]
0.49620836100000076
peak memory: 353.90 MiB, increment: 0.00 MiB


Pearson correlation features - normalized data

In [33]:
%memit DTClassifier(X_p_train_smote_COMBINED_N, y_p_train_smote, X_p_test_COMBINED_N, y_p_test)

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      3112
           1       0.52      0.64      0.57       587

    accuracy                           0.85      3699
   macro avg       0.73      0.76      0.74      3699
weighted avg       0.86      0.85      0.86      3699

[[2770  342]
 [ 213  374]]
0.21775195600000075
peak memory: 353.90 MiB, increment: 0.00 MiB


In [34]:
%memit RFClassifier(X_p_train_smote_COMBINED_N, y_p_train_smote, X_p_test_COMBINED_N, y_p_test)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      3112
           1       0.62      0.74      0.67       587

    accuracy                           0.89      3699
   macro avg       0.78      0.83      0.80      3699
weighted avg       0.90      0.89      0.89      3699

[[2847  265]
 [ 154  433]]
3.336997882999995
peak memory: 353.91 MiB, increment: 0.00 MiB


In [35]:
%memit LRClassifier(X_p_train_smote_COMBINED_N, y_p_train_smote, X_p_test_COMBINED_N, y_p_test)

              precision    recall  f1-score   support

           0       0.92      0.88      0.90      3112
           1       0.48      0.62      0.54       587

    accuracy                           0.83      3699
   macro avg       0.70      0.75      0.72      3699
weighted avg       0.85      0.83      0.84      3699

[[2723  389]
 [ 223  364]]
0.19465730099999234
peak memory: 353.91 MiB, increment: 0.00 MiB


Random Forest features - normalized data

In [36]:
%memit DTClassifier(X_rf_train_smote_COMBINED_N, y_rf_train_smote, X_rf_test_COMBINED_N, y_rf_test)

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      3110
           1       0.53      0.62      0.57       589

    accuracy                           0.85      3699
   macro avg       0.73      0.76      0.74      3699
weighted avg       0.86      0.85      0.86      3699

[[2786  324]
 [ 222  367]]
0.22273303000000055
peak memory: 353.91 MiB, increment: 0.00 MiB


In [37]:
%memit RFClassifier(X_rf_train_smote_COMBINED_N, y_rf_train_smote, X_rf_test_COMBINED_N, y_rf_test)

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      3110
           1       0.66      0.69      0.67       589

    accuracy                           0.89      3699
   macro avg       0.80      0.81      0.81      3699
weighted avg       0.90      0.89      0.89      3699

[[2898  212]
 [ 181  408]]
2.5844461160000094
peak memory: 353.91 MiB, increment: 0.00 MiB


In [38]:
%memit LRClassifier(X_rf_train_smote_COMBINED_N, y_rf_train_smote, X_rf_test_COMBINED_N, y_rf_test)

              precision    recall  f1-score   support

           0       0.91      0.94      0.92      3110
           1       0.61      0.49      0.54       589

    accuracy                           0.87      3699
   macro avg       0.76      0.72      0.73      3699
weighted avg       0.86      0.87      0.86      3699

[[2921  189]
 [ 298  291]]
0.31924783199998785
peak memory: 353.91 MiB, increment: 0.00 MiB


Recursive Feature Elimination - normalized data

In [39]:
%memit DTClassifier(X_rfe_train_smote_COMBINED_N, y_rfe_train_smote, X_rfe_test_COMBINED_N, y_rfe_test)

              precision    recall  f1-score   support

           0       0.87      0.75      0.80      3102
           1       0.24      0.42      0.30       597

    accuracy                           0.69      3699
   macro avg       0.55      0.58      0.55      3699
weighted avg       0.77      0.69      0.72      3699

[[2317  785]
 [ 349  248]]
0.13981943900000715
peak memory: 353.91 MiB, increment: 0.00 MiB


In [40]:
%memit RFClassifier(X_rfe_train_smote_COMBINED_N, y_rfe_train_smote, X_rfe_test_COMBINED_N, y_rfe_test)

              precision    recall  f1-score   support

           0       0.88      0.78      0.83      3102
           1       0.29      0.46      0.35       597

    accuracy                           0.73      3699
   macro avg       0.58      0.62      0.59      3699
weighted avg       0.79      0.73      0.75      3699

[[2417  685]
 [ 322  275]]
2.230331288000002
peak memory: 373.25 MiB, increment: 19.34 MiB


In [41]:
%memit LRClassifier(X_rfe_train_smote_COMBINED_N, y_rfe_train_smote, X_rfe_test_COMBINED_N, y_rfe_test)

              precision    recall  f1-score   support

           0       0.93      0.60      0.73      3102
           1       0.27      0.78      0.40       597

    accuracy                           0.63      3699
   macro avg       0.60      0.69      0.57      3699
weighted avg       0.83      0.63      0.68      3699

[[1851 1251]
 [ 131  466]]
0.23683189199999788
peak memory: 356.69 MiB, increment: 0.01 MiB


Data Standardization

In [42]:
sscaler = StandardScaler()

#Scaling control
X_train_smote_NUM_S = sscaler.fit_transform(X_train_smote_NUM)
X_train_smote_COMBINED_S = np.concatenate((X_train_smote_NUM_S, X_train_smote[cat_features]), axis = 1)

X_test_NUM_S = sscaler.transform(X_test_NUM)
X_test_COMBINED_S = np.concatenate((X_test_NUM_S, X_test[cat_features]), axis = 1)

#Scaling Pearson correlation features
X_p_train_smote_NUM_S = sscaler.fit_transform(X_p_train_smote_NUM)
X_p_train_smote_COMBINED_S = np.concatenate((X_p_train_smote_NUM_S, X_p_train_smote[p_cat_features]), axis = 1)

X_p_test_NUM_S = sscaler.transform(X_p_test_NUM)
X_p_test_COMBINED_S = np.concatenate((X_p_test_NUM_S, X_p_test[p_cat_features]), axis = 1)

#Scaling Random Forest features
X_rf_train_smote_NUM_S = sscaler.fit_transform(X_rf_train_smote_NUM)
X_rf_train_smote_COMBINED_S = np.concatenate((X_rf_train_smote_NUM_S, X_rf_train_smote[rf_cat_features]), axis = 1)

X_rf_test_NUM_S = sscaler.transform(X_rf_test_NUM)
X_rf_test_COMBINED_S = np.concatenate((X_rf_test_NUM_S, X_rf_test[rf_cat_features]), axis = 1)

#Scaling Recursive Feature Elimination features
X_rfe_train_smote_NUM_S = sscaler.fit_transform(X_rfe_train_smote_NUM)
X_rfe_train_smote_COMBINED_S = np.concatenate((X_rfe_train_smote_NUM_S, X_rfe_train_smote[rfe_cat_features]), axis = 1)

X_rfe_test_NUM_S = sscaler.transform(X_rfe_test_NUM)
X_rfe_test_COMBINED_S = np.concatenate((X_rfe_test_NUM_S, X_rfe_test[rfe_cat_features]), axis = 1)
%memit RFClassifier(X_train_smote_COMBINED_S, y_train_smote, X_test_COMBINED_S, y_test)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3137
           1       0.67      0.67      0.67       562

    accuracy                           0.90      3699
   macro avg       0.81      0.80      0.81      3699
weighted avg       0.90      0.90      0.90      3699

[[2956  181]
 [ 188  374]]
3.061092303999999
peak memory: 376.74 MiB, increment: 20.05 MiB


In [43]:
%memit RFClassifier(X_p_train_smote_COMBINED_S, y_p_train_smote, X_p_test_COMBINED_S, y_p_test)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      3112
           1       0.62      0.71      0.66       587

    accuracy                           0.88      3699
   macro avg       0.78      0.81      0.80      3699
weighted avg       0.89      0.88      0.89      3699

[[2852  260]
 [ 169  418]]
2.5837347960000017
peak memory: 376.74 MiB, increment: 0.00 MiB


In [44]:
%memit RFClassifier(X_rf_train_smote_COMBINED_S, y_rf_train_smote, X_rf_test_COMBINED_S, y_rf_test)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3110
           1       0.67      0.69      0.68       589

    accuracy                           0.90      3699
   macro avg       0.80      0.81      0.81      3699
weighted avg       0.90      0.90      0.90      3699

[[2909  201]
 [ 185  404]]
2.5904957519999954
peak memory: 367.74 MiB, increment: 14.02 MiB


Control data - Standardized data

In [45]:
%memit DTClassifier(X_train_smote_COMBINED_S, y_train_smote, X_test_COMBINED_S, y_test)

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3137
           1       0.54      0.62      0.57       562

    accuracy                           0.86      3699
   macro avg       0.73      0.76      0.75      3699
weighted avg       0.87      0.86      0.86      3699

[[2838  299]
 [ 215  347]]
0.28551911999998936
peak memory: 367.74 MiB, increment: 0.00 MiB


In [46]:
%memit RFClassifier(X_train_smote_COMBINED_S, y_train_smote, X_test_COMBINED_S, y_test)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3137
           1       0.67      0.65      0.66       562

    accuracy                           0.90      3699
   macro avg       0.80      0.80      0.80      3699
weighted avg       0.90      0.90      0.90      3699

[[2953  184]
 [ 196  366]]
2.4525311940000023
peak memory: 376.55 MiB, increment: 8.81 MiB


In [47]:
%memit LRClassifier(X_train_smote_COMBINED_S, y_train_smote, X_test_COMBINED_S, y_test)

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      3137
           1       0.75      0.41      0.53       562

    accuracy                           0.89      3699
   macro avg       0.82      0.69      0.73      3699
weighted avg       0.88      0.89      0.87      3699

[[3059   78]
 [ 334  228]]
0.49734107499999425
peak memory: 376.55 MiB, increment: 0.00 MiB


Pearson Correlation

In [48]:
%memit DTClassifier(X_p_train_smote_COMBINED_S, y_p_train_smote, X_p_test_COMBINED_S, y_p_test)

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      3112
           1       0.52      0.64      0.57       587

    accuracy                           0.85      3699
   macro avg       0.73      0.76      0.74      3699
weighted avg       0.86      0.85      0.86      3699

[[2772  340]
 [ 214  373]]
0.21296215899999993
peak memory: 376.56 MiB, increment: 0.01 MiB


In [49]:
%memit RFClassifier(X_p_train_smote_COMBINED_S, y_p_train_smote, X_p_test_COMBINED_S, y_p_test)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      3112
           1       0.62      0.72      0.67       587

    accuracy                           0.88      3699
   macro avg       0.78      0.82      0.80      3699
weighted avg       0.89      0.88      0.89      3699

[[2847  265]
 [ 162  425]]
3.540131871
peak memory: 376.56 MiB, increment: 0.00 MiB


In [50]:
%memit LRClassifier(X_p_train_smote_COMBINED_S, y_p_train_smote, X_p_test_COMBINED_S, y_p_test)

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      3112
           1       0.54      0.66      0.59       587

    accuracy                           0.86      3699
   macro avg       0.74      0.78      0.75      3699
weighted avg       0.87      0.86      0.86      3699

[[2784  328]
 [ 200  387]]
0.11496835799999872
peak memory: 376.56 MiB, increment: 0.00 MiB


Random Forest features - standardized data

In [51]:
%memit DTClassifier(X_rf_train_smote_COMBINED_S, y_rf_train_smote, X_rf_test_COMBINED_S, y_rf_test)

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      3110
           1       0.54      0.63      0.58       589

    accuracy                           0.86      3699
   macro avg       0.74      0.77      0.75      3699
weighted avg       0.87      0.86      0.86      3699

[[2799  311]
 [ 217  372]]
0.2193604289999911
peak memory: 376.56 MiB, increment: 0.00 MiB


In [52]:
%memit RFClassifier(X_rf_train_smote_COMBINED_S, y_rf_train_smote, X_rf_test_COMBINED_S, y_rf_test)

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      3110
           1       0.66      0.69      0.67       589

    accuracy                           0.89      3699
   macro avg       0.80      0.81      0.80      3699
weighted avg       0.89      0.89      0.89      3699

[[2898  212]
 [ 185  404]]
3.300864838999999
peak memory: 376.56 MiB, increment: 0.00 MiB


In [53]:
%memit LRClassifier(X_rf_train_smote_COMBINED_S, y_rf_train_smote, X_rf_test_COMBINED_S, y_rf_test)

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      3110
           1       0.64      0.54      0.59       589

    accuracy                           0.88      3699
   macro avg       0.78      0.74      0.76      3699
weighted avg       0.87      0.88      0.88      3699

[[2932  178]
 [ 268  321]]
0.25233621099999937
peak memory: 376.56 MiB, increment: 0.00 MiB


Recursive Feature Elimination features - standardized data

In [54]:
%memit DTClassifier(X_rfe_train_smote_COMBINED_S, y_rfe_train_smote, X_rfe_test_COMBINED_S, y_rfe_test)

              precision    recall  f1-score   support

           0       0.87      0.74      0.80      3102
           1       0.24      0.42      0.31       597

    accuracy                           0.69      3699
   macro avg       0.55      0.58      0.55      3699
weighted avg       0.77      0.69      0.72      3699

[[2300  802]
 [ 345  252]]
0.1390289379999956
peak memory: 376.57 MiB, increment: 0.00 MiB


In [55]:
%memit RFClassifier(X_rfe_train_smote_COMBINED_S, y_rfe_train_smote, X_rfe_test_COMBINED_S, y_rfe_test)

              precision    recall  f1-score   support

           0       0.88      0.78      0.83      3102
           1       0.28      0.46      0.35       597

    accuracy                           0.73      3699
   macro avg       0.58      0.62      0.59      3699
weighted avg       0.78      0.73      0.75      3699

[[2409  693]
 [ 324  273]]
2.7504613509999984
peak memory: 396.04 MiB, increment: 19.47 MiB


In [56]:
%memit LRClassifier(X_rfe_train_smote_COMBINED_S, y_rfe_train_smote, X_rfe_test_COMBINED_S, y_rfe_test)

              precision    recall  f1-score   support

           0       0.93      0.60      0.73      3102
           1       0.27      0.78      0.40       597

    accuracy                           0.63      3699
   macro avg       0.60      0.69      0.57      3699
weighted avg       0.83      0.63      0.68      3699

[[1864 1238]
 [ 133  464]]
0.24170232599999508
peak memory: 379.61 MiB, increment: 0.00 MiB
